Script done in collaboration with [Jean Spezia](http://invibe.net/LaurentPerrinet/JeanSpezia).
<!-- TEASER_END -->



In [1]:
import os

In [2]:
import numpy as np
import MotionClouds as mc
size = 5
mc.N_X, mc.N_Y, mc.N_frame = 2**size, 2**size, 2**size
fx, fy, ft = mc.get_grids(mc.N_X, mc.N_Y, mc.N_frame)
name = 'MotionPlaid'
mc.figpath = '../files/2015-02-19_MotionPlaid'
if not(os.path.isdir(mc.figpath)): os.mkdir(mc.figpath)

In [3]:
!mkdir -p results

In [4]:
#mask = 1 - np.abs(2*fx) - np.abs(2*fy)
#mask = 1.*(mask>0.)
#mask *= mask>0.
mask = 1 - np.max((np.abs(2*fx), np.abs(2*fy)), axis=0)
#mask = np.sqrt((2*fx)**2 + (2*fy)**2)
#mask = np.ones_like(fx)
print (mask.min(), mask.max(), mask.shape)
mov_=mc.anim_save(mask, os.path.join(mc.figpath, 'mask2'))
mc.in_show_video('mask2', figpath=mc.figpath)

0.0 1.0 (32, 32, 32)


""


In [5]:
N_orient, N_overlap = 8, 2
B_theta = 1.

In [6]:
mov = mc.np.zeros(((N_orient*mc.N_X), (N_orient*mc.N_X), mc.N_frame))
mov_mask = mc.np.zeros(((N_orient*mc.N_X), (N_orient*mc.N_X), mc.N_frame))
for i, theta1 in enumerate(np.linspace(0, 2*np.pi, N_orient*N_overlap, endpoint=False)[::-1]):
    diag1 = mc.envelope_gabor(fx, fy, ft, theta=theta1, V_X=np.cos(theta1), V_Y=np.sin(theta1), B_theta=B_theta)
    for j, theta2 in enumerate(np.linspace(0, 2*np.pi, N_orient*N_overlap, endpoint=False)):
        mov_ = np.zeros_like(mov)
        mov_mask_ = np.zeros_like(mov)
        diag2 = mc.envelope_gabor(fx, fy, ft, theta=theta2, V_X=np.cos(theta2), V_Y=np.sin(theta2), B_theta=B_theta)
        #mov[(i)*mc.N_X:(i+1)*mc.N_X, (j)*mc.N_Y : (j+1)*mc.N_Y, :] = mc.random_cloud(diag1 + diag2, seed=1234)
        mov_[:mc.N_X, :mc.N_Y, :] = mc.random_cloud(diag1 + diag2, seed=1234)*mask
        mov_mask_[:mc.N_X, :mc.N_Y, :] = mask
        #mov_[:mc.N_X, :mc.N_Y, :] = mask
        mov_ = np.roll(mov_, i*mc.N_X//N_overlap, axis=0)
        mov_ = np.roll(mov_, j*mc.N_Y//N_overlap, axis=1)
        mov += mov_
        mov_mask_ = np.roll(mov_mask_, i*mc.N_X//N_overlap, axis=0)
        mov_mask_ = np.roll(mov_mask_, j*mc.N_Y//N_overlap, axis=1)
        #print mov_mask_.max(), mov_mask_.min()
        mov_mask += mov_mask_

In [7]:
print (mov.max(), mov.min())

0.021823749347671922 -0.02810640379575316


In [8]:
mov_=mc.anim_save(mc.rectif(mov, contrast=.99), os.path.join(mc.figpath, name))
mc.in_show_video(name, figpath=mc.figpath)

""


In [9]:
mov_=mc.anim_save(mc.rectif(mov_mask, contrast=.99), os.path.join(mc.figpath, name))
mc.in_show_video(name, figpath=mc.figpath)

""


In [10]:
mov_=mc.anim_save(mc.rectif(mov/mov_mask, contrast=.99), os.path.join(mc.figpath, name))
mc.in_show_video(name, figpath=mc.figpath)

""


In [11]:
print (mov_mask.max(), mov_mask.min())

2.0 1.0


In [12]:
A = np.random.rand(5, 5)


In [13]:
mc.rectif(A, contrast=1, verbose=True)

Before Rectification of the frames
Mean= 0.43978454786297305 , std= 0.2720926860968015 , Min= 0.01025970733568149 , Max= 0.9813959661071936  Abs(Max)= 0.9813959661071936
After Rectification of the frames
Mean= 0.5000000000000001 , std= 0.2511880999286013 , Min= 0.10347508743472195 , Max= 1.0
percentage pixels clipped= 0.0


array([[0.76293304, 0.49006422, 0.5631343 , 0.37241039, 0.12623763],
       [0.67414584, 0.42895541, 0.62363661, 0.10927369, 0.31543963],
       [0.51993205, 0.19865564, 0.46936419, 0.426361  , 0.10347509],
       [0.80148186, 0.98015382, 0.22723519, 0.60546607, 0.41806369],
       [0.51916894, 0.25848294, 0.69983334, 1.        , 0.80609542]])

In [14]:
A

array([[0.72459963, 0.42902188, 0.50817307, 0.30157657, 0.03491662],
       [0.6284233 , 0.36282742, 0.57371054, 0.01654088, 0.23986455],
       [0.4613754 , 0.11336145, 0.40659914, 0.3600171 , 0.01025971],
       [0.76635659, 0.95989813, 0.14431948, 0.5540278 , 0.35102927],
       [0.46054878, 0.17816775, 0.65624859, 0.98139597, 0.77135409]])

In [15]:
N_orient, N_overlap = 8, 3
B_theta = 1.

In [16]:
%%timeit -n1

mov = mc.np.zeros(((N_orient*mc.N_X), (N_orient*mc.N_X), mc.N_frame))

for i, theta1 in enumerate(np.linspace(0, 2*np.pi, N_orient*N_overlap, endpoint=False)[::-1]):
    diag1 = mc.envelope_gabor(fx, fy, ft, theta=theta1, V_X=np.cos(theta1), V_Y=np.sin(theta1), B_theta=B_theta)
    for j, theta2 in enumerate(np.linspace(0, 2*np.pi, N_orient*N_overlap, endpoint=False)):
        mov_ = np.zeros_like(mov)
        diag2 = mc.envelope_gabor(fx, fy, ft, theta=theta2, V_X=np.cos(theta2), V_Y=np.sin(theta2), B_theta=B_theta)
        #mov[(i)*mc.N_X:(i+1)*mc.N_X, (j)*mc.N_Y : (j+1)*mc.N_Y, :] = mc.random_cloud(diag1 + diag2, seed=1234)
        mov_[:mc.N_X, :mc.N_Y, :] = mc.random_cloud(diag1 + diag2, seed=1234)
        #mov_[:mc.N_X, :mc.N_Y, :] = mask
        mov_ = np.roll(mov_, i*mc.N_X//N_overlap, axis=0)
        mov_ = np.roll(mov_, j*mc.N_Y//N_overlap, axis=1)
        mov += mov_

39.6 s ± 3.11 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit -n1

mov = mc.np.zeros(((N_orient*mc.N_X), (N_orient*mc.N_X), mc.N_frame))

for i, theta1 in enumerate(np.linspace(0, 2*np.pi, N_orient*N_overlap, endpoint=False)[::-1]):
    diag1 = mc.envelope_gabor(fx, fy, ft, theta=theta1, V_X=np.cos(theta1), V_Y=np.sin(theta1), B_theta=B_theta)
    mov_ = np.zeros_like(mov)
    mov_[:mc.N_X, :mc.N_Y, :] = mc.random_cloud(diag1, seed=1234)
    for j, theta2 in enumerate(np.linspace(0, 2*np.pi, N_orient*N_overlap, endpoint=False)):
        mov += np.roll(mov_, j*mc.N_Y//N_overlap, axis=1)
for j, theta2 in enumerate(np.linspace(0, 2*np.pi, N_orient*N_overlap, endpoint=False)):
    diag2 = mc.envelope_gabor(fx, fy, ft, theta=theta2, V_X=np.cos(theta2), V_Y=np.sin(theta2), B_theta=B_theta)
    mov_ = np.zeros_like(mov)
    mov_[:mc.N_X, :mc.N_Y, :] = mc.random_cloud(diag2, seed=1234)
    for i, theta1 in enumerate(np.linspace(0, 2*np.pi, N_orient*N_overlap, endpoint=False)[::-1]):
        mov += np.roll(mov_, i*mc.N_X//N_overlap, axis=0)


25.4 s ± 703 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit -n1

mov = mc.np.zeros(((N_orient*mc.N_X), (N_orient*mc.N_X), mc.N_frame))
mov1 = mc.np.zeros(((N_orient*mc.N_X), (N_orient*mc.N_X), mc.N_frame))
mov2 = mc.np.zeros(((N_orient*mc.N_X), (N_orient*mc.N_X), mc.N_frame))

for i, theta1 in enumerate(np.linspace(0, 2*np.pi, N_orient*N_overlap, endpoint=False)[::-1]):
    diag1 = mc.envelope_gabor(fx, fy, ft, theta=theta1, V_X=np.cos(theta1), V_Y=np.sin(theta1), B_theta=B_theta)
    mov_ = np.zeros_like(mov)
    mov_[:mc.N_X, :mc.N_Y, :] = mc.random_cloud(diag1, seed=1234)
    mov1 += np.roll(mov_, i*mc.N_X//N_overlap, axis=0)
    mov2 += np.roll(mov_, i*mc.N_Y//N_overlap, axis=1)
    for j, theta2 in enumerate(np.linspace(0, 2*np.pi, N_orient*N_overlap, endpoint=False)):
        mov1 += np.roll(mov1, j*mc.N_Y//N_overlap, axis=1)
        mov2 += np.roll(mov2, j*mc.N_Y//N_overlap, axis=0)

mov = mov1 + mov2

27.7 s ± 3.49 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%%timeit -n1

diag1 = mc.envelope_gabor(fx, fy, ft)


9.49 ms ± 3.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## some book keeping for the notebook

In [20]:
%load_ext version_information
%version_information numpy, scipy, matplotlib, MotionClouds

Software versions
Python 3.7.1 64bit [Clang 10.0.0 (clang-1000.11.45.5)]
IPython 7.1.1
OS Darwin 17.7.0 x86_64 i386 64bit
numpy 1.15.4
scipy 1.1.0
matplotlib 3.0.1
MotionClouds 20180606
Wed Nov 14 11:49:50 2018 CET